
### Logs
* 30.08: use 2048 for max_len and max_position_embeddings
* 03.09: use 512 instead
* 23.09: 512 pretrained with both datasets in MLM. training with 2048 and try to infer with 512. Pretraining notebook: pretrain1_task_ver3(512_larger dataset)
* 25.09 512 pretrained with both datasets in MLM. training with 1536. Pretraining notebook: pretrain1_task_ver3(512_larger dataset)
* no pretraining
* no pretraining use max_length as 1600, learning rate change to 2e-5
* no pretraining use max_length as 1600, learning rate changed to 4e-6, cosine scheduler
* no pretraining use max_legnth as 1600, learning rate changed back to 2e-6, cosine scheduler
* change weight decay from 0.01 to 1e-4, learning rate changed to 4e-6
* pretraining 1009, weight decay 1e-4, learning reate 4e-6
* pretraining 2109, weight decay 1e-4, learning reate 4e-6
* batch_size 8, eval batch_size 16

In [1]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install datasets -q
!pip install accelerate -U -q

In [2]:
# import pandas as pd
# new_train_data = [
#     pd.read_csv('./input/new_input_data.csv'),
#     pd.read_csv('./input/new_input_data_2.csv')
# ]
# new_train_data = pd.concat(new_train_data, axis = 0).reset_index(drop = True)

In [3]:
# new_train_data.sample(int(len(new_train_data)/5)).shape

In [4]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

from datasets import Dataset, disable_progress_bar
import pandas as pd

# pdf = pd.read_csv("./input/prompts_train.csv")
# sdf = pd.read_csv("./input/summaries_train.csv")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/miniconda3/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from transformers import AutoModelForSequenceClassification
#model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-large")
## use the pretrained model
# from transformers import AutoConfig
# config = AutoConfig.from_pretrained('./input/pretrain/pretrained_model/')
# model = AutoModelForSequenceClassification.from_pretrained('./input/pretrain/pretrained_model', config = config)

## Train

In [6]:
%%writefile train.py

import os
import logging
import warnings
from dataclasses import dataclass, field
from typing import Optional

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    set_seed,
    Trainer,
    TrainingArguments,
    HfArgumentParser,
    DataCollatorWithPadding,
)
from datasets import Dataset, disable_progress_bar
import pandas as pd
import numpy as np

warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['WANDB_PROJECT'] = 'kaggle-commonlit-eval-student-summaries-0910-pretrain-new-data'

disable_progress_bar()

@dataclass
class Config:
    model_name_or_path: Optional[str] = field(
        default="microsoft/deberta-v3-large",
        metadata={"help": "Model name or path"},
    )

    data_dir: Optional[str] = field(
        default="/kaggle/input/commonlit-evaluate-student-summaries",
        metadata={"help": "Data directory"},
    )

    max_seq_length: Optional[int] = field(
        default=1600,
        #default = 512,
        #default = 512,
        metadata={"help": "Max sequence length"},
    )

    add_prompt_question: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt question into input"},
    )

    add_prompt_text: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt text into input"},
    )

    fold: Optional[int] = field(
        default=0,
        metadata={"help": "Fold"},
    )

    num_proc: Optional[int] = field(
        default=4,
        metadata={"help": "Number of processes"},
    )

    dropout: Optional[float] = field(
        default=0.,
        metadata={"help": "Amount of dropout to apply"},
    )
    max_position_embeddings: Optional[int] = field(
        default=1600,
        #default=512,
        #default = 512,
        metadata={"help": "Amount of dropout to apply"},
    )


# Spell auto correction
# from spellchecker import SpellChecker

# def correct_spelling(input_text):
#     print('input_text: ', input_text)
#     # Initialize the spell checker
#     spell = SpellChecker()
#     # Split the input text into words
#     words = input_text.split()
#     # Initialize an empty list to store the corrected words
#     corrected_words = []
#     for word in words:
#         # Check if the word has any punctuation at the end
#         if word[-1].isalpha():
#             # Extract the punctuation
#             punctuation_end = ""
#         else:
#             punctuation_end = word[-1]
#             word = word[:-1]
#         # check if the word has any punctuation at the start
#         if word[0].isalpha():
#            # Extract the punctuation
#             punctuation_start = ""
#         else:
#             punctuation_start = word[0]
#             word = word[1:]
#         # Check the spelling of the word (case insensitive)
#         corrected_word = spell.correction(word.lower())
#         # Preserve the original capitalization
#         if word[0].isupper():
#             corrected_word = corrected_word.capitalize()
#         # Combine the corrected word and punctuation (if any)
#         corrected_word = punctuation_start+corrected_word+punctuation_end
#         # Append the corrected word to the list
#         corrected_words.append(corrected_word)
#     # Join the corrected words back into a single string
#     corrected_text = " ".join(corrected_words)
#     return corrected_text

def tokenize_pretrain(example, tokenizer, config):
    sep = tokenizer.sep_token

    # if config.add_prompt_question:
    #     text = sep.join(
    #         [example["prompt_question"], example["prompt_text"], example["text"]]
    #     )
    # elif config.add_prompt_text:
    #     text = sep.join([example["prompt_text"], example["text"]])
    # else:
    #     text = example["text"]
    #prompt = sep.join([example["prompt_title"], example["prompt_text"], example["prompt_question"]])
    labels = [example["content"], example["wording"]]

    tokenized = tokenizer(
#         prompt,
#         example["text"],
        example['summary'],
        example['input_text'],
        padding=False,
        truncation=True, # changed by Peng, turn on the truncation
        max_length=config.max_seq_length,
    )

    return {
        **tokenized,
        "labels": labels,
    }




def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }


def main():
    parser = HfArgumentParser((Config, TrainingArguments))

    config, training_args = parser.parse_args_into_dataclasses()

    set_seed(training_args.seed)

    if "wandb" in training_args.report_to:
        import wandb

        try:
#             from kaggle_secrets import UserSecretsClient
#             user_secrets = UserSecretsClient()
#             key = user_secrets.get_secret("wandb")

#             wandb.login(key=key)
            wandb.login()
        except:
            print("Could not log in to WandB")

    tokenizer = AutoTokenizer.from_pretrained(config.model_name_or_path)
    model_config = AutoConfig.from_pretrained(config.model_name_or_path)

    model_config.update({
        "hidden_dropout_prob": config.dropout,
        "attention_probs_dropout_prob": config.dropout,
        "num_labels": 2,
        "problem_type": "regression",
        "max_position_embeddings": config.max_position_embeddings,
        "cfg": config.__dict__,
    })

    print(model_config)
    
    # Do not use pretrained model
    model = AutoModelForSequenceClassification.from_pretrained(
        config.model_name_or_path, config=model_config
    )
    
    # use pretrained model
    #print('use pretrained_model')
    #print('./input/pretrain/pretrained_model_2109/')
    #model = AutoModelForSequenceClassification.from_pretrained('./input/pretrain/pretrained_model_2109/', config = model_config)

    # read pretrain data
    new_train_data = [
        pd.read_csv('./input/new_input_data.csv'),
        pd.read_csv('./input/new_input_data_2.csv')
    ]
    new_train_data = pd.concat(new_train_data, axis = 0).reset_index(drop = True)
    new_train_data = new_train_data.sample(int(len(new_train_data)/5)).reset_index(drop = True)
    print('shape of pretrain data: ', new_train_data.shape);
    
    ## Create dataset
    pretrain_ds = Dataset.from_pandas(new_train_data)

    pretrain_ds = pretrain_ds.map(
        tokenize_pretrain,
        batched=False,
        num_proc=config.num_proc,
        fn_kwargs={"tokenizer": tokenizer, "config": config},
    )

#     val_ds = val_ds.map(
#         tokenize,
#         batched=False,
#         num_proc=config.num_proc,
#         fn_kwargs={"tokenizer": tokenizer, "config": config},
#     )

    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        pad_to_multiple_of=16 if training_args.fp16 else None,
    )

    training_args.bf16 =True
    training_args.gradient_accumulation_steps = 1
    training_args.load_best_model_at_end = True
    training_args.greater_is_better = False
    training_args.metric_for_best_model = 'mcrmse'
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=pretrain_ds,
        eval_dataset=pretrain_ds,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_mcrmse,
    )

    trainer.train()

    model.config.best_metric = trainer.state.best_metric
    model.config.save_pretrained(training_args.output_dir)

    trainer.log({"eval_best_mcrmse": trainer.state.best_metric})


if __name__ == "__main__":
    main()

Overwriting train.py


In [7]:
from pathlib import Path

seed = 42

fold = 1

output = f"pretrain_seed{seed}_0910"

!python train.py \
  --model_name_or_path "microsoft/deberta-v3-large" \
  --add_prompt_question True \
  --fold $fold \
  --data_dir "./" \
  --output_dir $output \
  --fp16 \
  --num_train_epochs 4 \
  --dataloader_num_workers 4 \
  --learning_rate 4e-6 \
  --weight_decay 1e-4 \
  --warmup_ratio 0 \
  --lr_scheduler_type "cosine" \
  --optim "adamw_torch" \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2 \
  --evaluation_strategy "steps" \
  --eval_steps 150 \
  --save_strategy "steps" \
  --save_steps 150 \
  --save_total_limit 1 \
  --report_to "wandb" \
  --metric_for_best_model "mcrmse" \
  --greater_is_better False \
  --logging_steps 10 \
  --log_level "error" \
  --disable_tqdm True \
  --ddp_find_unused_parameters False \
  --dropout 0 \
  --seed $seed


output_dir = Path.cwd() / output
# add json files
for json_file in output_dir.glob("checkpoint*/*token*.json"):
    json_file.rename(output_dir/json_file.name)

# model files
for model_file in output_dir.glob("checkpoint*/*model*"):
    model_file.rename(output_dir/model_file.name)

# remove optimizer states and other files
to_delete = str(list(output_dir.glob("checkpoint*"))[0])
!rm -r $to_delete

wandb: Currently logged in as: peng_sun. Use `wandb login --relogin` to force relogin
DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "attention_probs_dropout_prob": 0.0,
  "cfg": {
    "add_prompt_question": true,
    "add_prompt_text": false,
    "data_dir": "./",
    "dropout": 0.0,
    "fold": 1,
    "max_position_embeddings": 1600,
    "max_seq_length": 1600,
    "model_name_or_path": "microsoft/deberta-v3-large",
    "num_proc": 4
  },
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 1600,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
 

{'loss': 0.0236, 'learning_rate': 3.996068607986581e-06, 'epoch': 0.08}
{'loss': 0.1464, 'learning_rate': 3.9959597469910834e-06, 'epoch': 0.08}
{'loss': 0.0147, 'learning_rate': 3.9958494008411575e-06, 'epoch': 0.08}
{'eval_loss': 0.027143551036715508, 'eval_content_rmse': 0.15525087714195251, 'eval_wording_rmse': 0.1737365424633026, 'eval_mcrmse': 0.16449370980262756, 'eval_runtime': 619.2756, 'eval_samples_per_second': 29.404, 'eval_steps_per_second': 14.703, 'epoch': 0.08}
{'loss': 0.0266, 'learning_rate': 3.995737569618912e-06, 'epoch': 0.08}
{'loss': 0.0238, 'learning_rate': 3.995624253407558e-06, 'epoch': 0.08}
{'loss': 0.0155, 'learning_rate': 3.995509452291411e-06, 'epoch': 0.09}
{'loss': 0.0293, 'learning_rate': 3.995393166355893e-06, 'epoch': 0.09}
{'loss': 0.026, 'learning_rate': 3.995275395687529e-06, 'epoch': 0.09}
{'loss': 0.0259, 'learning_rate': 3.9951561403739515e-06, 'epoch': 0.09}
{'loss': 0.0179, 'learning_rate': 3.995035400503894e-06, 'epoch': 0.09}
{'loss': 0.038

{'loss': 0.0069, 'learning_rate': 3.979849934839594e-06, 'epoch': 0.18}
{'eval_loss': 0.03397902846336365, 'eval_content_rmse': 0.1854596734046936, 'eval_wording_rmse': 0.18320152163505554, 'eval_mcrmse': 0.18433059751987457, 'eval_runtime': 625.6606, 'eval_samples_per_second': 29.104, 'eval_steps_per_second': 14.553, 'epoch': 0.18}
{'loss': 0.02, 'learning_rate': 3.979604921888385e-06, 'epoch': 0.18}
{'loss': 0.0517, 'learning_rate': 3.979358435952053e-06, 'epoch': 0.18}
{'loss': 0.0146, 'learning_rate': 3.979110477214003e-06, 'epoch': 0.18}
{'loss': 0.0202, 'learning_rate': 3.978861045858738e-06, 'epoch': 0.19}
{'loss': 0.0528, 'learning_rate': 3.978610142071853e-06, 'epoch': 0.19}
{'loss': 0.018, 'learning_rate': 3.978357766040042e-06, 'epoch': 0.19}
{'loss': 0.025, 'learning_rate': 3.978103917951092e-06, 'epoch': 0.19}
{'loss': 0.0275, 'learning_rate': 3.977848597993887e-06, 'epoch': 0.19}
{'loss': 0.0252, 'learning_rate': 3.977591806358406e-06, 'epoch': 0.19}
{'loss': 0.0205, 'lea

{'loss': 0.0206, 'learning_rate': 3.9515848224271034e-06, 'epoch': 0.28}
{'loss': 0.0235, 'learning_rate': 3.951206796801303e-06, 'epoch': 0.28}
{'loss': 0.0184, 'learning_rate': 3.950827319320866e-06, 'epoch': 0.28}
{'loss': 0.0141, 'learning_rate': 3.950446390268156e-06, 'epoch': 0.28}
{'loss': 0.0127, 'learning_rate': 3.950064009926612e-06, 'epoch': 0.29}
{'loss': 0.0108, 'learning_rate': 3.949680178580758e-06, 'epoch': 0.29}
{'loss': 0.0349, 'learning_rate': 3.949294896516195e-06, 'epoch': 0.29}
{'loss': 0.012, 'learning_rate': 3.9489081640196045e-06, 'epoch': 0.29}
{'loss': 0.0185, 'learning_rate': 3.948519981378745e-06, 'epoch': 0.29}
{'loss': 0.0259, 'learning_rate': 3.948130348882455e-06, 'epoch': 0.29}
{'loss': 0.0127, 'learning_rate': 3.947739266820654e-06, 'epoch': 0.29}
{'loss': 0.0056, 'learning_rate': 3.947346735484339e-06, 'epoch': 0.29}
{'loss': 0.0194, 'learning_rate': 3.9469527551655826e-06, 'epoch': 0.29}
{'loss': 0.0376, 'learning_rate': 3.94655732615754e-06, 'epoch

{'loss': 0.0213, 'learning_rate': 3.910767467320009e-06, 'epoch': 0.38}
{'loss': 0.016, 'learning_rate': 3.910257188042489e-06, 'epoch': 0.38}
{'loss': 0.0163, 'learning_rate': 3.909745487380132e-06, 'epoch': 0.38}
{'loss': 0.081, 'learning_rate': 3.909232365713686e-06, 'epoch': 0.39}
{'loss': 0.0531, 'learning_rate': 3.908717823424952e-06, 'epoch': 0.39}
{'loss': 0.0287, 'learning_rate': 3.9082018608967916e-06, 'epoch': 0.39}
{'loss': 0.0061, 'learning_rate': 3.907684478513124e-06, 'epoch': 0.39}
{'loss': 0.009, 'learning_rate': 3.907165676658922e-06, 'epoch': 0.39}
{'loss': 0.0164, 'learning_rate': 3.906645455720216e-06, 'epoch': 0.39}
{'loss': 0.0088, 'learning_rate': 3.906123816084093e-06, 'epoch': 0.39}
{'loss': 0.0223, 'learning_rate': 3.905600758138695e-06, 'epoch': 0.39}
{'loss': 0.0236, 'learning_rate': 3.905076282273217e-06, 'epoch': 0.39}
{'loss': 0.0176, 'learning_rate': 3.904550388877912e-06, 'epoch': 0.4}
{'eval_loss': 0.027430459856987, 'eval_content_rmse': 0.16730055212

...

{'eval_loss': 0.015842542052268982, 'eval_content_rmse': 0.11168789863586426, 'eval_wording_rmse': 0.13860303163528442, 'eval_mcrmse': 0.12514546513557434, 'eval_runtime': 642.3252, 'eval_samples_per_second': 28.349, 'eval_steps_per_second': 14.175, 'epoch': 0.99}
{'loss': 0.0149, 'learning_rate': 3.4263598165858963e-06, 'epoch': 0.99}
{'loss': 0.1212, 'learning_rate': 3.425270986420423e-06, 'epoch': 0.99}
{'loss': 0.0092, 'learning_rate': 3.424060167663637e-06, 'epoch': 0.99}
{'loss': 0.0126, 'learning_rate': 3.4228482892916633e-06, 'epoch': 0.99}
{'loss': 0.0158, 'learning_rate': 3.421635352206237e-06, 'epoch': 0.99}
{'loss': 0.0184, 'learning_rate': 3.4204213573098804e-06, 'epoch': 1.0}
{'loss': 0.0643, 'learning_rate': 3.4192063055059033e-06, 'epoch': 1.0}
{'loss': 0.0054, 'learning_rate': 3.417990197698402e-06, 'epoch': 1.0}
{'loss': 0.0099, 'learning_rate': 3.416773034792258e-06, 'epoch': 1.0}
{'loss': 0.0081, 'learning_rate': 3.415554817693139e-06, 'epoch': 1.0}
{'loss': 0.011, 

{'loss': 0.022, 'learning_rate': 3.3134629638348866e-06, 'epoch': 1.09}
{'loss': 0.0361, 'learning_rate': 3.3121614583185792e-06, 'epoch': 1.09}
{'loss': 0.0249, 'learning_rate': 3.310858976448714e-06, 'epoch': 1.09}
{'loss': 0.0123, 'learning_rate': 3.3095555191944438e-06, 'epoch': 1.09}
{'loss': 0.0048, 'learning_rate': 3.308251087525645e-06, 'epoch': 1.09}
{'loss': 0.0235, 'learning_rate': 3.3069456824129184e-06, 'epoch': 1.09}
{'loss': 0.0033, 'learning_rate': 3.305639304827592e-06, 'epoch': 1.1}
{'loss': 0.0097, 'learning_rate': 3.3043319557417144e-06, 'epoch': 1.1}
{'loss': 0.0043, 'learning_rate': 3.303023636128059e-06, 'epoch': 1.1}
{'loss': 0.0292, 'learning_rate': 3.301714346960121e-06, 'epoch': 1.1}
{'loss': 0.0111, 'learning_rate': 3.300404089212116e-06, 'epoch': 1.1}
{'loss': 0.0079, 'learning_rate': 3.2990928638589816e-06, 'epoch': 1.1}
{'loss': 0.023, 'learning_rate': 3.2977806718763745e-06, 'epoch': 1.1}
{'loss': 0.0073, 'learning_rate': 3.296467514240671e-06, 'epoch': 

{'loss': 0.0216, 'learning_rate': 3.1898998081119014e-06, 'epoch': 1.19}
{'loss': 0.0079, 'learning_rate': 3.1885127125153844e-06, 'epoch': 1.19}
{'loss': 0.0066, 'learning_rate': 3.1871247325699133e-06, 'epoch': 1.19}
{'loss': 0.0048, 'learning_rate': 3.1857358693082546e-06, 'epoch': 1.19}
{'loss': 0.009, 'learning_rate': 3.184346123763837e-06, 'epoch': 1.19}
{'loss': 0.0078, 'learning_rate': 3.1829554969707406e-06, 'epoch': 1.19}
{'loss': 0.0108, 'learning_rate': 3.1815639899637044e-06, 'epoch': 1.2}
{'loss': 0.0044, 'learning_rate': 3.1801716037781205e-06, 'epoch': 1.2}
{'loss': 0.0074, 'learning_rate': 3.1787783394500373e-06, 'epoch': 1.2}
{'loss': 0.0094, 'learning_rate': 3.1773841980161554e-06, 'epoch': 1.2}
{'loss': 0.008, 'learning_rate': 3.175989180513827e-06, 'epoch': 1.2}
{'loss': 0.0072, 'learning_rate': 3.1745932879810577e-06, 'epoch': 1.2}
{'loss': 0.0198, 'learning_rate': 3.1731965214565045e-06, 'epoch': 1.2}
{'eval_loss': 0.008303587324917316, 'eval_content_rmse': 0.092

{'loss': 0.0055, 'learning_rate': 3.058716621911633e-06, 'epoch': 1.29}
{'loss': 0.0069, 'learning_rate': 3.0572525697469456e-06, 'epoch': 1.29}
{'loss': 0.016, 'learning_rate': 3.0557877309013966e-06, 'epoch': 1.29}
{'loss': 0.0141, 'learning_rate': 3.0543221064649432e-06, 'epoch': 1.29}
{'loss': 0.0115, 'learning_rate': 3.052855697528128e-06, 'epoch': 1.29}
{'loss': 0.007, 'learning_rate': 3.0513885051820776e-06, 'epoch': 1.3}
{'loss': 0.0207, 'learning_rate': 3.0499205305184993e-06, 'epoch': 1.3}
{'loss': 0.0062, 'learning_rate': 3.0484517746296863e-06, 'epoch': 1.3}
{'loss': 0.0199, 'learning_rate': 3.0469822386085105e-06, 'epoch': 1.3}
{'loss': 0.0074, 'learning_rate': 3.045511923548424e-06, 'epoch': 1.3}
{'loss': 0.0077, 'learning_rate': 3.044040830543459e-06, 'epoch': 1.3}
{'eval_loss': 0.010590129531919956, 'eval_content_rmse': 0.09753336012363434, 'eval_wording_rmse': 0.1080159991979599, 'eval_mcrmse': 0.10277467966079712, 'eval_runtime': 648.2178, 'eval_samples_per_second': 2

{'loss': 0.0077, 'learning_rate': 2.9208692466667673e-06, 'epoch': 1.39}
{'loss': 0.0416, 'learning_rate': 2.919337453544768e-06, 'epoch': 1.39}
{'loss': 0.096, 'learning_rate': 2.9178049763618348e-06, 'epoch': 1.39}
{'loss': 0.0127, 'learning_rate': 2.916271816258253e-06, 'epoch': 1.39}
{'loss': 0.0152, 'learning_rate': 2.914737974374818e-06, 'epoch': 1.4}
{'loss': 0.0074, 'learning_rate': 2.913203451852831e-06, 'epoch': 1.4}
{'loss': 0.0152, 'learning_rate': 2.9116682498341004e-06, 'epoch': 1.4}
{'loss': 0.0033, 'learning_rate': 2.91013236946094e-06, 'epoch': 1.4}
{'loss': 0.0083, 'learning_rate': 2.9085958118761668e-06, 'epoch': 1.4}
^C
Traceback (most recent call last):
  File "train.py", line 268, in <module>
    main()
  File "train.py", line 259, in main
    trainer.train()
  File "/root/miniconda3/lib/python3.8/site-packages/transformers/trainer.py", line 1539, in train
    return inner_training_loop(
  File "/root/miniconda3/lib/python3.8/site-packages/transformers/trainer.py"

In [18]:
from pathlib import Path
import json

seed = 42
scores = []
for fold in range(0,4):
    output = f"./output_fold{fold}_seed{seed}_0810"
    p = Path(output) / "config.json"

    with open(p) as fp:
        cfg = json.load(fp)


    print(f"MCRMSE for fold {fold}, seed {seed}: {cfg['best_metric']:.4f}")
    scores.append(cfg['best_metric'])
print(sum(scores)/len(scores))

MCRMSE for fold 0, seed 42: 0.5513
MCRMSE for fold 1, seed 42: 0.4587
MCRMSE for fold 2, seed 42: 0.5038
MCRMSE for fold 3, seed 42: 0.4316
0.4863356798887253


#df.loc[0, 'prompt_text']

In [18]:
# from pathlib import Path
# import json

# seed = 42
# scores = []
# for fold in range(4):
#     output = f"./output_fold{fold}_seed{seed}_3008"
#     p = Path(output) / "config.json"

#     with open(p) as fp:
#         cfg = json.load(fp)


#     print(f"MCRMSE for fold {fold}, seed {seed}: {cfg['best_metric']:.4f}")
#     scores.append(cfg['best_metric'])
# print(sum(scores)/len(scores))